In [11]:
import pandas as pd

# Begin by loading in zpids dataset
zpids_units = pd.read_csv('D:/DataSets/msc_group_project/property_keys_units.csv')
zpids_non_units = pd.read_csv('D:/DataSets/msc_group_project/property_keys_non_units.csv')

zpids_df = zpids_units + zpids_non_units

zpids_df = pd.concat([zpids_units,zpids_non_units], ignore_index=True)

#save new dataframe 

#save it
import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/all_property_keys.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
zpids_df.to_csv(save_path, index=False)

In [348]:
len(zpids_df)

2836

In [268]:
response2 = retrive_property_details(90005746)

In [293]:
# create function that sends out a request
import requests

def retrive_property_details(zpid):

    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpid}

    headers = {
        "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()



In [244]:
response = response1


In [284]:
response = response2
zpids_df["zpids"][i]

2075342017

In [233]:
len(response["resoFacts"]["utilities"])

1

In [294]:
def check_feature(check_str, res, return_statement):
    if res is not None and check_str in res:
        if isinstance(res[check_str], dict) and return_statement in res[check_str]:
            return res[check_str][return_statement]
    return None

In [334]:
import time
# create loop that makes a request for each property
# each of these lists represent a table that will be in our schema

new_row_main_details = []
new_row_addresses = []
new_row_features = [] # includes window features, associated amenities, utilities, lot features, parking, patio and porch
new_row_exterior_features = []
new_row_at_a_glance = []
new_row_fees = []
new_row_price_history = []
new_row_schools_nearby = []

iterations = len(zpids_df["zpids"])
for i in range(iterations):
    print("iteration : " + str(i) + ", zpid = " + str(zpids_df["zpids"][i]))
    
    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpids_df["zpids"][i]}

    headers = {
        "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring).json()

        if response is not None:
            # a temportary main information dictionary we create just for this iteration
            temp_main_details = {}
            temp_main_details["zpid"] = response.get("zpid")
            temp_main_details["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_main_details["buildingName"] = check_feature("building",response,"buildingName")

            temp_main_details["postingGroupName"] = response["listingProvider"]["postingGroupName"]
            temp_main_details["agentName"] = response["listingProvider"]["agentName"]
            temp_main_details["overview"] = response["description"]
            temp_main_details["url"] = response["url"]
            temp_main_details["timeOnHME"] = response["timeOnZillow"]
            temp_main_details["price"] = response["price"]
            temp_main_details["lvingAreaValue"] = response["livingAreaValue"]       
            temp_main_details["bathrooms"] = response["resoFacts"]["bathrooms"]
            temp_main_details["bathroomsHalf"] = response["resoFacts"]["bathroomsHalf"]
            temp_main_details["bathroomsFull"] = response["resoFacts"]["bathroomsFull"]
            temp_main_details["bedrooms"] = response["resoFacts"]["bedrooms"]
            temp_main_details["livingArea"] = response["resoFacts"]["livingArea"]
            temp_main_details["livingAreaValue"] = response["livingAreaValue"]
            temp_main_details["stories"] = response["resoFacts"]["stories"]
            temp_main_details["yearBuilt"] = response["resoFacts"]["yearBuilt"]
            temp_main_details["homeType"] = response["resoFacts"]["homeType"]
            temp_main_details["homeStatus"] = response["homeStatus"]
            temp_main_details["datePosted"] = response["datePosted"]
            temp_main_details["rentEstimate"] = response["rentZestimate"]

            # a temportary addresses dictionary we create just for this iteration
            temp_addresses = {}
            temp_addresses["zpid"] = response["zpid"]
            temp_addresses["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_addresses["buildingName"] = check_feature("building",response,"buildingName")

            temp_addresses["city"] = response["address"].get("city")
            temp_addresses["state"] = response["address"].get("state")
            temp_addresses["streetAddress"] = response["address"].get("streetAddress")
            temp_addresses["zipcode"] = response["address"].get("zipcode")
            temp_addresses["cityRegion"] = response["resoFacts"].get("cityRegion")

            temp_addresses["stateId"] = response.get("stateId")
            temp_addresses["cityId"] = response.get("cityId")
            temp_addresses["country"] = response.get("country")
            temp_addresses["zillowBuildingUrl"] = str("https:/zillow.com/") + str(check_feature("building",response,"bdpUrl"))

            # a temporary features information dictionary we create just for this iteration
            temp_features = {}
            count = 0
            temp_features["zpid"] = response["zpid"]
            temp_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("appliances"):
                for i in range(len(response["resoFacts"]["appliances"])):
                    temp_features["feature " + str(count) ] = response["resoFacts"]["appliances"][i]
                    count += 1
            if response["resoFacts"].get("lotFeatures"):
                for i in range(len(response["resoFacts"]["lotFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["lotFeatures"][i]
                    count += 1
            if response["resoFacts"].get("communityFeatures"):
                for i in range(len(response["resoFacts"]["communityFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["communityFeatures"][i]
                    count += 1
            if  response["resoFacts"].get("utilities"):
                for i in range(len(response["resoFacts"]["utilities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["utilities"][i]
                    count += 1
            if response["resoFacts"].get("windowFeatures"):
                for i in range(len(response["resoFacts"]["windowFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["windowFeatures"][i]
                    count += 1
            if response["resoFacts"].get("laundryFeatures"):
                for i in range(len(response["resoFacts"]["laundryFeatures"])):
                    temp_features["feature" + str(count)] = "Laundry Features: " + response["resoFacts"]["laundryFeatures"][i]
                    count += 1
            if response["resoFacts"].get("patioAndPorchFeatures"):
                for i in range(len(response["resoFacts"]["patioAndPorchFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["patioAndPorchFeatures"][i]
                    count += 1
            if response["resoFacts"].get("associationAmenities"):
                for i in range(len(response["resoFacts"]["associationAmenities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["associationAmenities"][i]
                    count += 1

            # a temporary exterior features information dictionary we create just for this iteration
            temp_exterior_features = {}
            temp_exterior_features["zpid"] = response["zpid"]
            temp_exterior_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_exterior_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("exteriorFeatures"):
                for i in range(len(response["resoFacts"]["exteriorFeatures"])):
                    temp_exterior_features["exterior feature " + str(i)] = response["resoFacts"]["exteriorFeatures"][i]

            # a temporary at a glance information dictionary we create just for this iteration
            temp_atAGlance = {}
            temp_atAGlance["zpid"] = response["zpid"]
            temp_atAGlance["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_atAGlance["buildingName"] = check_feature("building",response,"buildingName")
            if response["resoFacts"].get("atAGlanceFacts"):
                for prop in response["resoFacts"]["atAGlanceFacts"]:
                    temp_atAGlance[str(prop["factLabel"])] = str(prop["factValue"])

            # a temporary fees information dictionary we create just for this iteration
            temp_fees = {}
            temp_fees["zpid"] = response["zpid"]
            temp_fees["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_fees["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("feesAndDues"):
                for prop in response["resoFacts"]["feesAndDues"]:
                    temp_fees[str(prop["type"] + " fee")] = prop["fee"]

            # a temporary price history information dictionary we create just for this iteration
            temp_price_history = {}
            temp_price_history["zpid"] = response["zpid"]
            temp_price_history["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_price_history["buildingName"] = check_feature("building",response,"buildingName")
            if response.get("priceHistory"):
                for index, prop in enumerate(response["priceHistory"]):
                    temp_price_history["change_" + str(index) + "_date"] = prop["date"]
                    temp_price_history["change_" + str(index) + "_priceChangeRate"] = prop["priceChangeRate"]
                    temp_price_history["change_" + str(index) + "_postingIsRental"] = prop["postingIsRental"]
                    temp_price_history["change_" + str(index) + "_time"] = prop["time"]
                    temp_price_history["change_" + str(index) + "_event"] = prop["event"]
                    temp_price_history["change_" + str(index) + "_price"] = prop["price"]

            # a temporary school nearby information dictionary we create just for this iteration
            temp_schools_nearby = {}
            temp_schools_nearby["zpid"] = response["zpid"]
            temp_schools_nearby["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_schools_nearby["buildingName"] = check_feature("building",response,"buildingName")

            if response.get("schools"):
                for index, prop in enumerate(response["schools"]):
                    temp_schools_nearby["school_" + str(index) + "_name"] = prop["name"]
                    temp_schools_nearby["school_" + str(index) + "_rating"] = prop["rating"]
                    temp_schools_nearby["school_" + str(index) + "_distance"] = prop["distance"]
                    temp_schools_nearby["school_" + str(index) + "_link"] = prop["link"]
                    temp_schools_nearby["school_" + str(index) + "_level"] = prop["level"]
                    temp_schools_nearby["school_" + str(index) + "_grades"] = prop["grades"]
                    temp_schools_nearby["school_" + str(index) + "_type"] = prop["type"]

        else:
            # Handle the case where the JSON response is not as expected
            error_message = "Invalid JSON response"

    except Exception as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred during the request: {str(e)}")
        
    new_row_main_details.append(temp_main_details)
    new_row_addresses.append(temp_addresses)
    new_row_features.append(temp_features)
    new_row_exterior_features.append(temp_exterior_features)
    new_row_at_a_glance.append(temp_atAGlance)
    new_row_fees.append(temp_fees)
    new_row_price_history.append(temp_price_history)
    new_row_schools_nearby.append(temp_schools_nearby)
    
    
        # If we have made 1 requests and it's not the last iteration, sleep for 2 seconds
    time.sleep(1)
    
main_details_df = pd.DataFrame(new_row_main_details)
address_details_df = pd.DataFrame(new_row_addresses)
indoor_features_df = pd.DataFrame(new_row_features)
exterior_features_df = pd.DataFrame(new_row_exterior_features)
at_a_glance_df = pd.DataFrame(new_row_at_a_glance)
fees_df = pd.DataFrame(new_row_fees)
price_history_df = pd.DataFrame(new_row_price_history)
schools_nearby_df = pd.DataFrame(new_row_schools_nearby)

pd.set_option('display.max_columns', None)
 

iteration : 0, zpid = 2075342017
iteration : 1, zpid = 2075330457
iteration : 2, zpid = 2069661399
iteration : 3, zpid = 2068739347
iteration : 4, zpid = 2057534299
iteration : 5, zpid = 2075652105
iteration : 6, zpid = 2075119575
iteration : 7, zpid = 2067797682
iteration : 8, zpid = 2071152602
iteration : 9, zpid = 2075652104
iteration : 10, zpid = 2072105027
iteration : 11, zpid = 2062003721
iteration : 12, zpid = 2078692687
iteration : 13, zpid = 2078284715
iteration : 14, zpid = 2056426687
iteration : 15, zpid = 2057597584
iteration : 16, zpid = 2055783116
iteration : 17, zpid = 2079758736
iteration : 18, zpid = 2061338347
iteration : 19, zpid = 2078217448
iteration : 20, zpid = 2079746270
iteration : 21, zpid = 2058365311
iteration : 22, zpid = 2055464704
iteration : 23, zpid = 2078657076
iteration : 24, zpid = 2055587413
iteration : 25, zpid = 2055587412
iteration : 26, zpid = 2057558335
iteration : 27, zpid = 2057564784
iteration : 28, zpid = 2055222389
iteration : 29, zpid = 2

iteration : 238, zpid = 2059925680
iteration : 239, zpid = 2059930142
iteration : 240, zpid = 2059930144
iteration : 241, zpid = 2059930161
iteration : 242, zpid = 2059930163
iteration : 243, zpid = 2059930110
iteration : 244, zpid = 2059930114
iteration : 245, zpid = 2059930176
iteration : 246, zpid = 2055658242
iteration : 247, zpid = 2059930120
iteration : 248, zpid = 2057807563
iteration : 249, zpid = 2057807580
iteration : 250, zpid = 2059930172
iteration : 251, zpid = 2057807562
iteration : 252, zpid = 2057807589
iteration : 253, zpid = 2059930105
iteration : 254, zpid = 2059930106
iteration : 255, zpid = 2059930121
iteration : 256, zpid = 2059930170
iteration : 257, zpid = 2059930160
iteration : 258, zpid = 2057912892
iteration : 259, zpid = 2059930162
iteration : 260, zpid = 2059930159
iteration : 261, zpid = 2057807571
iteration : 262, zpid = 2084886858
iteration : 263, zpid = 2089864371
iteration : 264, zpid = 2089864376
iteration : 265, zpid = 2089864352
iteration : 266, zpi

iteration : 473, zpid = 2057092979
iteration : 474, zpid = 2055999715
iteration : 475, zpid = 2057281772
iteration : 476, zpid = 2055821446
iteration : 477, zpid = 2055824351
iteration : 478, zpid = 2057281722
iteration : 479, zpid = 2057281823
iteration : 480, zpid = 2057281829
iteration : 481, zpid = 2057281825
iteration : 482, zpid = 2057281816
iteration : 483, zpid = 2057281804
iteration : 484, zpid = 2057281784
iteration : 485, zpid = 2057281720
iteration : 486, zpid = 2057281778
iteration : 487, zpid = 2057281710
iteration : 488, zpid = 2057281763
iteration : 489, zpid = 2057281728
iteration : 490, zpid = 2057281771
iteration : 491, zpid = 2057281712
iteration : 492, zpid = 2057281708
iteration : 493, zpid = 2057281704
iteration : 494, zpid = 2057281699
iteration : 495, zpid = 2057281706
iteration : 496, zpid = 2057281700
iteration : 497, zpid = 2057281740
iteration : 498, zpid = 2057281732
iteration : 499, zpid = 2057281721
iteration : 500, zpid = 2057281811
iteration : 501, zpi

iteration : 708, zpid = 2057079585
iteration : 709, zpid = 2066963462
iteration : 710, zpid = 2055893857
iteration : 711, zpid = 2057550673
iteration : 712, zpid = 2061865597
iteration : 713, zpid = 2065798859
iteration : 714, zpid = 2065798864
iteration : 715, zpid = 2066963481
iteration : 716, zpid = 2079760556
iteration : 717, zpid = 2066382168
iteration : 718, zpid = 2079760591
iteration : 719, zpid = 2068622578
iteration : 720, zpid = 2063383716
iteration : 721, zpid = 2072068928
iteration : 722, zpid = 2071737831
iteration : 723, zpid = 2069673601
iteration : 724, zpid = 2062831265
iteration : 725, zpid = 2067663716
iteration : 726, zpid = 2056930246
iteration : 727, zpid = 2070354029
iteration : 728, zpid = 2079760588
iteration : 729, zpid = 2056072972
iteration : 730, zpid = 2079760573
iteration : 731, zpid = 2067192153
iteration : 732, zpid = 2072068922
iteration : 733, zpid = 2079760585
iteration : 734, zpid = 2079760562
iteration : 735, zpid = 2072068914
iteration : 736, zpi

iteration : 943, zpid = 2067039647
iteration : 944, zpid = 2067039651
iteration : 945, zpid = 2067039703
iteration : 946, zpid = 2067053058
iteration : 947, zpid = 2056465329
iteration : 948, zpid = 2060174234
iteration : 949, zpid = 2062673555
iteration : 950, zpid = 2067050282
iteration : 951, zpid = 2066823013
iteration : 952, zpid = 2063461017
iteration : 953, zpid = 2066477096
iteration : 954, zpid = 2067050256
iteration : 955, zpid = 2055236062
iteration : 956, zpid = 2056967805
iteration : 957, zpid = 2067050292
iteration : 958, zpid = 2067050293
iteration : 959, zpid = 2067050294
iteration : 960, zpid = 2067050290
iteration : 961, zpid = 2064297626
iteration : 962, zpid = 2067039709
iteration : 963, zpid = 2062658302
iteration : 964, zpid = 2065616697
iteration : 965, zpid = 2058738329
iteration : 966, zpid = 2065739285
iteration : 967, zpid = 2062109374
iteration : 968, zpid = 2067050328
iteration : 969, zpid = 2061517154
iteration : 970, zpid = 2061517155
iteration : 971, zpi

...

iteration : 1841, zpid = 2090771673
iteration : 1842, zpid = 2094727958
iteration : 1843, zpid = 2091478066
iteration : 1844, zpid = 2094704554
iteration : 1845, zpid = 2094798273
iteration : 1846, zpid = 2062507258
iteration : 1847, zpid = 2057002880
iteration : 1848, zpid = 2063232972
iteration : 1849, zpid = 2056303543
iteration : 1850, zpid = 2064230484
iteration : 1851, zpid = 2061241356
iteration : 1852, zpid = 2062179376
iteration : 1853, zpid = 2056433757
iteration : 1854, zpid = 67870593
iteration : 1855, zpid = 2055480587
iteration : 1856, zpid = 67871091
iteration : 1857, zpid = 67870583
iteration : 1858, zpid = 67870594
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 1859, zpid = 67871084
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 1860, zpid = 72228374
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 1861, zpid = 67870625
iteration : 1862, zpid = 67

iteration : 2005, zpid = 2056132345
iteration : 2006, zpid = 2066875326
iteration : 2007, zpid = 2055580792
iteration : 2008, zpid = 2076776672
iteration : 2009, zpid = 2056285322
iteration : 2010, zpid = 2103606521
iteration : 2011, zpid = 2056367574
iteration : 2012, zpid = 2061785820
iteration : 2013, zpid = 2061783598
iteration : 2014, zpid = 2061783597
iteration : 2015, zpid = 2065711816
iteration : 2016, zpid = 2063311953
iteration : 2017, zpid = 2066398619
iteration : 2018, zpid = 2055440379
iteration : 2019, zpid = 2093874048
iteration : 2020, zpid = 2093476214
iteration : 2021, zpid = 2068513683
iteration : 2022, zpid = 2090666507
iteration : 2023, zpid = 2102431546
iteration : 2024, zpid = 2084507467
iteration : 2025, zpid = 2071097450
iteration : 2026, zpid = 2093041497
iteration : 2027, zpid = 2071571004
iteration : 2028, zpid = 2077402585
iteration : 2029, zpid = 2068470755
iteration : 2030, zpid = 2056017616
iteration : 2031, zpid = 2067936571
iteration : 2032, zpid = 206

iteration : 2233, zpid = 2060544072
iteration : 2234, zpid = 2060541888
iteration : 2235, zpid = 2060540490
iteration : 2236, zpid = 2056225154
iteration : 2237, zpid = 2071178396
iteration : 2238, zpid = 2083459505
iteration : 2239, zpid = 2098844235
iteration : 2240, zpid = 36470554
iteration : 2241, zpid = 36470490
iteration : 2242, zpid = 2071780953
iteration : 2243, zpid = 2087276245
iteration : 2244, zpid = 2057426147
iteration : 2245, zpid = 2057429198
iteration : 2246, zpid = 2062323916
iteration : 2247, zpid = 2090330964
iteration : 2248, zpid = 2056386548
iteration : 2249, zpid = 2084722420
iteration : 2250, zpid = 2056386549
iteration : 2251, zpid = 2057100410
iteration : 2252, zpid = 2056389350
iteration : 2253, zpid = 2057183597
iteration : 2254, zpid = 2099384584
iteration : 2255, zpid = 2057109254
iteration : 2256, zpid = 2066280650
iteration : 2257, zpid = 2091215207
iteration : 2258, zpid = 2092284525
iteration : 2259, zpid = 2056496661
iteration : 2260, zpid = 2056496

iteration : 2454, zpid = 241969634
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 2455, zpid = 2094522779
iteration : 2456, zpid = 36483299
iteration : 2457, zpid = 36536857
iteration : 2458, zpid = 36439291
iteration : 2459, zpid = 36594073
iteration : 2460, zpid = 2055522894
iteration : 2461, zpid = 36599196
iteration : 2462, zpid = 36491324
iteration : 2463, zpid = 36514623
iteration : 2464, zpid = 36430079
iteration : 2465, zpid = 36435864
iteration : 2466, zpid = 36560327
iteration : 2467, zpid = 2068668283
iteration : 2468, zpid = 2081380607
iteration : 2469, zpid = 36566347
iteration : 2470, zpid = 2068499299
iteration : 2471, zpid = 36434672
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 2472, zpid = 2055974613
iteration : 2473, zpid = 36585414
iteration : 2474, zpid = 2106499799
iteration : 2475, zpid = 36452421
iteration : 2476, zpid = 36477922
iteration : 2477, zpid = 36533901
An error occurr

iteration : 2670, zpid = 2066775014
iteration : 2671, zpid = 2055940693
iteration : 2672, zpid = 2094436221
iteration : 2673, zpid = 36477070
iteration : 2674, zpid = 36447679
iteration : 2675, zpid = 2079134499
iteration : 2676, zpid = 36630091
iteration : 2677, zpid = 2069181583
iteration : 2678, zpid = 2055984378
iteration : 2679, zpid = 36489569
iteration : 2680, zpid = 36514007
iteration : 2681, zpid = 67870832
An error occurred during the request: 'NoneType' object is not subscriptable
iteration : 2682, zpid = 2078689830
iteration : 2683, zpid = 2056012203
iteration : 2684, zpid = 36534419
iteration : 2685, zpid = 81750016
iteration : 2686, zpid = 36598505
iteration : 2687, zpid = 36511369
iteration : 2688, zpid = 2056033771
iteration : 2689, zpid = 36460596
iteration : 2690, zpid = 90008340
iteration : 2691, zpid = 36435095
iteration : 2692, zpid = 36599027
iteration : 2693, zpid = 36618454
iteration : 2694, zpid = 2056291487
iteration : 2695, zpid = 36434991
iteration : 2696, z

In [337]:
main_details_df

# save new dataframe 

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/main_information.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
main_details_df.to_csv(save_path, index=False)

In [338]:
address_details_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/addresses.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
address_details_df.to_csv(save_path, index=False)

In [340]:
indoor_features_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/interior_features.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
indoor_features_df.to_csv(save_path, index=False)

In [341]:
exterior_features_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/exterior_features.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
exterior_features_df.to_csv(save_path, index=False)

In [343]:
at_a_glance_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/quick_glance_features.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
at_a_glance_df.to_csv(save_path, index=False)

In [344]:
fees_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/all_fees.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
fees_df.to_csv(save_path, index=False)

In [345]:
price_history_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/price_history.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
price_history_df.to_csv(save_path, index=False)

In [349]:
schools_nearby_df

import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/nearby_schools.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
schools_nearby_df.to_csv(save_path, index=False)